In [15]:
"""
Generate descriptive SNBB figures that match the slide template colours.
--------------------------------------------------------------------------

Prereqs:
    pip install matplotlib pandas numpy seaborn
    (seaborn only used for its convenient styling helper)

Replace CSV_PATH if your file lives elsewhere.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from matplotlib import font_manager as fm

In [16]:
finished_subjects = pd.read_csv("/media/storage/phd/papers/aging/schaefer2018tian2020_400_7/subjects.csv").values.flatten().tolist()

In [17]:
SOFTWARE = "mrtrix3"
ATLAS = "schaefer2018tian2020_400_7"
RECONSTRUCTION = "SDStream"
SCALE = "None"
WEIGHT = "SIFT2"


DISTRIBUTION_METRIC = "qfmean"
DIFFUSION_MEASURES = ["adc", "fa", "rd", "ad"]
STRUCTURAL_MEASURES = ["gm_vol"]
CONNECTOME_MEASURES = [
    "degree",
    "eigenvector_centrality",
    "betweenness_centrality",
    "degree_norm",
    "regional_efficiency",
]

COLUMNS_TO_DROP = ['study', 'group',
       'condition', 'path', 'gender', 'sexual_orientation', 'marital_status',
       'handedness', 'relationship_duration', 'number_of_children',
       'exercise_frequency', 'measure', 'weight', 'height', 'education_level',
       'psychometric_score_total', 'psychometric_score_verbal',
       'psychometric_score_quantitative', 'psychometric_score_english',"dob","subject_id"]

In [18]:
destination = Path(f"/media/storage/phd/papers/aging/{ATLAS}")

# unique_measures = DIFFUSION_MEASURES + STRUCTURAL_MEASURES + CONNECTOME_MEASURES
unique_measures = DIFFUSION_MEASURES + STRUCTURAL_MEASURES


results = {m:{} for m in unique_measures}

for m in unique_measures:
    print(f"Processing {m}")
    # m = "degree"
    try:
        if m not in CONNECTOME_MEASURES:
            m_data = pd.read_pickle(destination / f"{m}.pkl").reset_index(
                drop=True
            )
        else:
            m_data = pd.read_pickle(
                destination / f"connectome_{RECONSTRUCTION}_{SCALE}_{WEIGHT}.pkl"
            ).reset_index(drop=True)
            m_data["measure"] = m
    except FileNotFoundError:
        raise FileNotFoundError(f"Could not find {m}.pkl. Skipping...")
    data = m_data.drop_duplicates(
        subset=["subject_code", "index"], keep="last"
    )
    data = data[data["age_at_scan"] > 18]
    data = data[data["subject_code"].isin(finished_subjects)]
    data = data.drop(columns=COLUMNS_TO_DROP)
    data.to_csv(f"/home/galkepler/Projects/neuroaging/data/processed/{m}.csv")
    # df = collect_data(m_data, m)
    # break

Processing adc
Processing fa
Processing rd
Processing ad
Processing gm_vol
